In [1]:
import pandas as pd
from be_great import GReaT

c:\Users\nbasa\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv("data/merged_df_file.csv")

In [3]:
def process_csv(df):

    df = df.dropna()
    '''for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]): # Handle numerical missing values (mean imputation)
            if df[col].isnull().any():
                df[col].fillna(df[col].mean(), inplace=True)
        else: # Handle missing values in categorical or other object columns
            if pd.api.types.is_object_dtype(df[col]):
                if df[col].isnull().any():
                    df[col].fillna(df[col].mode()[0], inplace=True)  # Impute with mode for categorical data'''
    return df

clean_data = process_csv(data)

In [4]:
clean_data = clean_data[(clean_data['numberRating'] >= 0) & (clean_data['numberRating'] <= 3)]
clean_data = clean_data[(clean_data['numberMessageReceived'] >= 0) & (clean_data['numberMessageReceived'] <= 3)]
clean_data = clean_data[(clean_data['numberMessageRead'] >= 0) & (clean_data['numberMessageRead'] <= 3)]
clean_data = clean_data[(clean_data['reward'] <= 2)]

In [5]:
# When generating at day_part_x  = 0, columns numberRating, numberMessageReceived and numberMessageRead have a max of 1, 
# for day_part_x  = 1 the max is 2 and for day_part_x  = 2 the max is 3.

def day_part_no_check(df):
    conditions = {
        0: {'numberRating': 1, 'numberMessageReceived': 1, 'numberMessageRead': 1},
        1: {'numberRating': 2, 'numberMessageReceived': 2, 'numberMessageRead': 2},
        2: {'numberRating': 3, 'numberMessageReceived': 3, 'numberMessageRead': 3}
    }
    
    for day_part, max_values in conditions.items():
        for col, max_value in max_values.items():
            df = df[~((df['day_part_x'] == day_part) & (df[col] > max_value))]
    
    return df

# Filter the DataFrame
clean_data = day_part_no_check(clean_data)

In [6]:
# Action can only take 0, 1, 2, same applies to day_part_x.

def check_action_daypart(df):
    # Define conditions
    condition_action = df['action'].isin([0, 1, 2])
    condition_day_part_x = df['day_part_x'].isin([0, 1, 2])
    
    # Filter DataFrame based on conditions
    df_filtered = df[condition_action & condition_day_part_x]
    
    return df_filtered

# Call the function
clean_data = check_action_daypart(clean_data)

In [ ]:
os.makedirs(os.path.dirname("data/clean_data.csv"), exist_ok=True)
clean_data.to_csv("data/clean_data.csv", index=False)

In [ ]:
model = GReaT(llm='distilgpt2', 
              batch_size=32,  
              epochs=1, 
              logging_steps=100)

model.fit(clean_data)

synthetic_data = model.sample(len(clean_data))

In [ ]:
import os

# Save synthetic data to CSV file
os.makedirs(os.path.dirname("data/great_data.csv"), exist_ok=True)
synthetic_data.to_csv("data/great_data.csv", index=False)